In [1]:
import numpy as np
import pandas as pd
import json
import collections
from collections import defaultdict 
from functools import partial
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import jax

# Global flag to set a specific platform, must be used at startup.
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_debug_nans", True)
jax.config.update("jax_debug_infs", True)

jax.config.update('jax_log_compiles', False)
jax.config.update('jax_check_tracer_leaks', False)

In [2]:
import jax.numpy as jnp
a = jnp.array(range(1000)) 
a.sum()

DeviceArray(499500, dtype=int32)

In [3]:
# Good read: https://iq-inc.com/importerror-attempted-relative-import/

import sys
import importlib
from mimicnet import concept
from mimicnet import jax_interface
from mimicnet import dag
from mimicnet import glove
from mimicnet import gram
from mimicnet import ode
from mimicnet import models

importlib.reload(sys.modules['mimicnet.concept'])
importlib.reload(sys.modules['mimicnet.dag'])
importlib.reload(sys.modules['mimicnet.jax_interface'])
importlib.reload(sys.modules['mimicnet.glove'])
importlib.reload(sys.modules['mimicnet.gram'])
importlib.reload(sys.modules['mimicnet.ode'])
importlib.reload(sys.modules['mimicnet.models'])

<module 'mimicnet.models' from '/home/asem/GP/MIMIC-SNONET/mimicnet/models.py'>

In [4]:

KG = dag.CCSDAG()

In [5]:
# multi_visit_mimic_dir = '/home/am8520/GP/ehr-data/mimic3-multi-visit'
multi_visit_mimic_dir = '/home/asem/GP/ehr-data/mimic3-multi-visit'
transformed_mimic_dir = '/home/asem/GP/ehr-data/mimic3-transforms'
mimic_dir = '/home/asem/GP/ehr-data/mimic3-v1.4/physionet.org/files/mimiciii/1.4'
# mimic_dir = '/home/asem/GP/MIMIC-SNONET/RAW/mimic-iii-clinical-database-1.4'

experiments_dir = '/home/asem/GP/ehr-data/mimic3-snonet-exp'
experiment_prefix = 'DEC03'

In [6]:
D_LABITEMS = pd.read_csv(f'{mimic_dir}/D_LABITEMS.csv.gz')
D_ITEMS = pd.read_csv(f'{mimic_dir}/D_ITEMS.csv.gz')
D_TEST = pd.concat([D_LABITEMS, D_ITEMS], join='inner')
test_label_dict = dict(zip(D_TEST.ITEMID, D_TEST.LABEL))
test_cat_dict = dict(zip(D_TEST.ITEMID, D_TEST.CATEGORY))

In [7]:
static_df = pd.read_csv(f'{transformed_mimic_dir}/static_df.csv.gz')
adm_df = pd.read_csv(f'{transformed_mimic_dir}/adm_df.csv.gz')
diag_df = pd.read_csv(f'{transformed_mimic_dir}/diag_df.csv.gz', dtype={'ICD9_CODE': str})
proc_df = pd.read_csv(f'{transformed_mimic_dir}/proc_df.csv.gz', dtype={'ICD9_CODE': str})
test_df = pd.read_csv(f'{transformed_mimic_dir}/test_df.csv.gz')


In [8]:
# Cast columns of dates to datetime64

static_df['DOB'] = pd.to_datetime(static_df.DOB, infer_datetime_format=True).dt.normalize()
adm_df['ADMITTIME'] = pd.to_datetime(adm_df.ADMITTIME, infer_datetime_format=True).dt.normalize()
adm_df['DISCHTIME'] = pd.to_datetime(adm_df.DISCHTIME, infer_datetime_format=True).dt.normalize()
test_df['DATE'] = pd.to_datetime(test_df.DATE, infer_datetime_format=True).dt.normalize()

In [9]:
static_df.ETHNIC_GROUP.nunique()

7

In [10]:
test_df

,SUBJECT_ID,ITEMID,DATE,VALUE
0,17,50852,2134-12-22,-1.201339
1,17,50861,2134-12-22,-0.809370
2,17,50862,2134-12-22,1.634410
3,17,50863,2134-12-22,-1.452596
4,17,50867,2134-12-22,-0.608894
...,...,...,...,...
4546204,99982,227456,2157-02-22,1.794272
4546205,99982,227457,2157-02-22,-0.723781
4546206,99982,227465,2157-02-22,2.247434
4546207,99982,227466,2157-02-22,-0.029395


In [11]:
patients = concept.Subject.to_list(static_df, adm_df, diag_df, proc_df, test_df)

In [12]:
len(patients)

4434

In [13]:
# ehr_dfs = concept.Subject.to_df(patients)

In [14]:
# def df_eq(df1, df2):
#     df11 = df1.sort_values(by=df1.columns.tolist()).reset_index(drop=True)
#     df22 = df2.sort_values(by=df2.columns.tolist()).reset_index(drop=True)
#     return df11[df22.columns].equals(df22)

# all(map(df_eq, ehr_dfs, [static_df, adm_df, diag_df, proc_df, test_df]))

### [FORK] Skip the cell below to load the jaxified data from a stored file on disc

In [15]:
subjects_interface = jax_interface.SubjectJAXInterface(patients, set(test_df.ITEMID), KG)
import pickle
with open(f'{experiments_dir}/{experiment_prefix}_subjects_interface.pkl', 'wb') as pickleFile:
    pickle.dump(subjects_interface, pickleFile)

In [16]:
# import pickle
# with open(f'{experiments_dir}/{experiment_prefix}_subjects_interface.pkl', 'rb') as pickleFile:
#     subjects_interface = pickle.load(pickleFile)

## GloVe Initialization

In [17]:
%load_ext autoreload
%autoreload 2

In [18]:
import logging
logging.basicConfig(level=logging.DEBUG)
logging.debug("test")

In [19]:
glove_args = {
    'diag_idx': subjects_interface.diag_multi_ccs_idx,
    'proc_idx': subjects_interface.proc_multi_ccs_idx,
    'ccs_dag': KG,
    'subjects': patients,
    'diag_vector_size': 100,
    'proc_vector_size': 60,
    'iterations': 30,
    'window_size_days': 2 * 365
}

diag_glove_rep, proc_glove_rep = glove.glove_representation(**glove_args)

In [20]:
print(f'#point_indices: {len(subjects_interface.nth_points)}')
print(f'#total_points: {sum(len(points) for n, points in subjects_interface.nth_points.items())}')

#[len(points) for n, points in subjects_interface.nth_points.items()]

#point_indices: 1085
#total_points: 129334


## GRAM objects

In [21]:
from datetime import datetime
daily_tracer = "/tmp/tensorboard/"+ datetime.now().strftime("%Y%m%d-%H%M%S") 
print(daily_tracer)

/tmp/tensorboard/20211204-184820


In [22]:


logs = '/tmp/tensorboard/20210708-182059'
#server = jax.profiler.start_server(9999)

In [29]:
       
config = {
    'gram_config': {
        'diag': {
            'ccs_dag': KG,
            'code2index': subjects_interface.diag_multi_ccs_idx,
            'attention_method': 'tanh', #l2, tanh
            'attention_dim': 50,
            'ancestors_mat': subjects_interface.diag_multi_ccs_ancestors_mat,
            'basic_embeddings': diag_glove_rep
        },
        'proc': {
            'ccs_dag': KG,
            'code2index': subjects_interface.proc_multi_ccs_idx,
            'attention_method': 'tanh',
            'attention_dim': 50,
            'ancestors_mat': subjects_interface.proc_multi_ccs_ancestors_mat,
            'basic_embeddings': proc_glove_rep
        }
    },
    'model': {
        'ode_dyn': 'mlp', # gru, mlp
        'state_size': 50,
        'numeric_hidden_size': 50,
        'bias': True
    },
    'training': {
        'train_validation_split': 0.8,
        'batch_size': 4,
        'epochs': 200,
        'lr': 1e-3,
        'diag_loss': 'balanced_focal', # balanced_focal, bce
        'tay_reg': 0, # Order of regularized derivative of the dynamics function (None for disable).
        'loss_mixing': {
            'num_alpha': 0.1,
            'diag_alpha': 0.1,
            'ode_alpha': 1e-3,
            'l1_reg': 1e-6,
            'l2_reg': 1e-5,
            'dyn_reg': 1e-5
        },
        'eval_freq': 10,
        'save_freq': 100,
        'save_params_prefix': None
    }
}


In [30]:
       
# config = {
#     'gram_config': {
#         'diag': {
#             'ccs_dag': KG,
#             'code2index': subjects_interface.diag_multi_ccs_idx,
#             'attention_method': 'tanh', #l2, tanh
#             'attention_dim': 150,
#             'ancestors_mat': subjects_interface.diag_multi_ccs_ancestors_mat,
#             'basic_embeddings': diag_glove_rep
#         },
#         'proc': {
#             'ccs_dag': KG,
#             'code2index': subjects_interface.proc_multi_ccs_idx,
#             'attention_method': 'tanh',
#             'attention_dim': 100,
#             'ancestors_mat': subjects_interface.proc_multi_ccs_ancestors_mat,
#             'basic_embeddings': proc_glove_rep
#         }
#     },
#     'model': {
#         'ode_dyn': 'mlp', # gru, mlp
#         'state_size': 120,
#         'numeric_hidden_size': 200,
#         'bias': True
#     },
#     'training': {
#         'train_validation_split': 0.8,
#         'batch_size': 20,
#         'epochs': 200,
#         'lr': 1e-3,
#         'diag_loss': 'balanced_focal', # balanced_focal, bce
#         'tay_reg': 2, # Order of regularized derivative of the dynamics function (None for disable).
#         'loss_mixing': {
#             'num_alpha': 0.1,
#             'diag_alpha': 0.1,
#             'ode_alpha': 1e-3,
#             'l1_reg': 1e-6,
#             'l2_reg': 1e-5,
#             'dyn_reg': 1e-5
#         },
#         'eval_freq': 10,
#         'save_freq': 100,
#         'save_params_prefix': None
#     }
# }


In [31]:
diag_gram = gram.DAGGRAM(**config['gram_config']['diag'])

In [32]:
proc_gram = gram.DAGGRAM(**config['gram_config']['proc'])

## GRU-ODE-Bayes

In [ ]:
import random
%load_ext autoreload
%autoreload 2
from absl import logging
logging.set_verbosity(logging.INFO)

#with jax.profiler.trace(logs):
res = ode.train_ehr(subject_interface=subjects_interface,
                diag_gram=diag_gram,
                proc_gram=proc_gram,
                rng=random.Random(42),
                **config['model'],
                **config['training'],
                verbose_debug=False,
                shape_debug=False,
                nan_debug=False,
                memory_profile=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


INFO:ode:#params: 295884
INFO:ode:shape(params): {'diag_gram': ((589, 100), FlatMapping({
  'None_DAG_Attention/~/linear': FlatMapping({'b': (50,), 'w': (200, 50)}),
  'None_DAG_Attention/~/linear_1': FlatMapping({'w': (50, 1)}),
})), 'f_dec': FlatMapping({
  'f_dec/~/lin_gram': FlatMapping({'b': (100,), 'w': (150, 100)}),
  'f_dec/~/lin_h_hidden': FlatMapping({'b': (50,), 'w': (50, 50)}),
  'f_dec/~/lin_num_hidden1': FlatMapping({'b': (50,), 'w': (550, 50)}),
  'f_dec/~/lin_num_hidden2': FlatMapping({'b': (100,), 'w': (50, 100)}),
  'f_dec/~/lin_out': FlatMapping({'b': (284,), 'w': (100, 284)}),
}), 'f_num': FlatMapping({
  'f_numeric/linear': FlatMapping({'b': (50,), 'w': (50, 50)}),
  'f_numeric/~/linear': FlatMapping({'b': (550,), 'w': (50, 550)}),
  'f_numeric/~/linear_1': FlatMapping({'b': (550,), 'w': (50, 550)}),
}), 'gru_bayes': FlatMapping({
  'gru_bayes/~/gru': FlatMapping({'b': (150,), 'w_h': (50, 150), 'w_i': (50, 150)}),
  'gru_bayes/~/gru_bayes_prep1': FlatMapping({'b': 

GRAM:   0%|          | 1/177350 [00:07<366:06:40,  7.43s/it]          

point #8/1084 - odeint:   0%|          | 1/177350 [00:07<367:11:50,  7.45s/it]

point #8/1084:   0%|          | 1/177350 [00:07<367:51:04,  7.47s/it]         

point #6/1084 - odeint:   0%|          | 1/177350 [06:26<19059:27:27, 386.89s/it]it]


point #8/1084 - pre_dag_loss:   0%|          | 1/177350 [00:10<501:51:52, 10.19s/it]

point #8/1084 - gru_bayes:   0%|          | 1/177350 [00:10<502:00:05, 10.19s/it]   

point #8/1084 - num_post_loss:   0%|          | 1/177350 [00:10<502:50:55, 10.21s/it]

point #8/1084 - dag_post_loss:   0%|          | 1/177350 [00:10<503:43:11, 10.22s/it]

point #9/1084:   0%|          | 1/177350 [00:10<503:54:55, 10.23s/it]                

info retrieval:   0%|          | 1/177350 [00:10<504:05:32, 10.23s/it]

GRAM:   0%|          | 1/177350 [00:10<504:20:33, 10.24s/it]          

point #9/1084 - odeint:   0%|          | 1/177350 [00:10<505:01:29, 10.25s/it]

point #9/1084:   0%|    

point #17/1084:   0%|          | 1/177350 [00:11<549:55:08, 11.16s/it]         

point #17/1084 - pre_num_loss:   0%|          | 1/177350 [00:11<549:59:46, 11.16s/it]

point #17/1084 - pre_dag_loss:   0%|          | 1/177350 [00:11<550:53:48, 11.18s/it]

point #17/1084 - gru_bayes:   0%|          | 1/177350 [00:11<551:46:07, 11.20s/it]   

point #17/1084 - num_post_loss:   0%|          | 1/177350 [00:11<552:36:30, 11.22s/it]

point #17/1084 - dag_post_loss:   0%|          | 1/177350 [00:11<553:39:23, 11.24s/it]

point #18/1084:   0%|          | 1/177350 [00:11<564:37:31, 11.46s/it]                

info retrieval:   0%|          | 1/177350 [00:11<564:41:52, 11.46s/it]

GRAM:   0%|          | 1/177350 [00:11<564:48:53, 11.47s/it]          

point #18/1084 - odeint:   0%|          | 1/177350 [00:11<565:29:11, 11.48s/it]

point #18/1084:   0%|          | 1/177350 [00:11<566:17:51, 11.50s/it]         

point #18/1084 - pre_num_loss:   0%|          | 1/177350 [00:11<566:25:47, 11.50s/it]

p

point #26/1084 - pre_num_loss:   0%|          | 1/177350 [00:12<628:58:27, 12.77s/it]

point #26/1084 - pre_dag_loss:   0%|          | 1/177350 [00:12<630:25:53, 12.80s/it]

point #26/1084 - gru_bayes:   0%|          | 1/177350 [00:12<630:35:17, 12.80s/it]   

point #26/1084 - num_post_loss:   0%|          | 1/177350 [00:12<631:22:22, 12.82s/it]

point #26/1084 - dag_post_loss:   0%|          | 1/177350 [00:12<632:22:59, 12.84s/it]

point #27/1084:   0%|          | 1/177350 [00:12<632:33:54, 12.84s/it]                

info retrieval:   0%|          | 1/177350 [00:12<632:43:03, 12.84s/it]

GRAM:   0%|          | 1/177350 [00:12<632:56:16, 12.85s/it]          

point #27/1084 - odeint:   0%|          | 1/177350 [00:12<633:41:26, 12.86s/it]

point #27/1084:   0%|          | 1/177350 [00:12<634:25:56, 12.88s/it]         

point #27/1084 - pre_num_loss:   0%|          | 1/177350 [00:12<634:35:19, 12.88s/it]

point #27/1084 - pre_dag_loss:   0%|          | 1/177350 [00:12<636:00:32, 12.91s/

...

point #878/1084:   0%|          | 1/177350 [00:23<1142:08:48, 23.18s/it]

info retrieval:   0%|          | 1/177350 [00:23<1142:23:14, 23.19s/it] 

point #879/1084:   0%|          | 1/177350 [00:23<1142:32:53, 23.19s/it]

info retrieval:   0%|          | 1/177350 [00:23<1142:39:38, 23.19s/it] 

point #880/1084:   0%|          | 1/177350 [00:23<1142:46:11, 23.20s/it]

info retrieval:   0%|          | 1/177350 [00:23<1142:52:22, 23.20s/it] 

point #881/1084:   0%|          | 1/177350 [00:23<1142:57:53, 23.20s/it]

info retrieval:   0%|          | 1/177350 [00:23<1143:04:25, 23.20s/it] 

point #882/1084:   0%|          | 1/177350 [00:23<1143:20:14, 23.21s/it]

info retrieval:   0%|          | 1/177350 [00:23<1143:29:17, 23.21s/it] 

point #883/1084:   0%|          | 1/177350 [00:23<1143:36:49, 23.21s/it]

info retrieval:   0%|          | 1/177350 [00:23<1143:44:43, 23.22s/it] 

point #884/1084:   0%|          | 1/177350 [00:23<1143:49:18, 23.22s/it]

info retrieval:   0%|          | 1/177

point #929/1084:   0%|          | 1/177350 [00:23<1155:34:16, 23.46s/it]

info retrieval:   0%|          | 1/177350 [00:23<1155:47:14, 23.46s/it] 

point #930/1084:   0%|          | 1/177350 [00:23<1155:56:17, 23.46s/it]

info retrieval:   0%|          | 1/177350 [00:23<1156:02:09, 23.47s/it] 

point #931/1084:   0%|          | 1/177350 [00:23<1156:08:14, 23.47s/it]

info retrieval:   0%|          | 1/177350 [00:23<1156:14:47, 23.47s/it] 

point #932/1084:   0%|          | 1/177350 [00:23<1156:26:50, 23.47s/it]

info retrieval:   0%|          | 1/177350 [00:23<1156:32:02, 23.48s/it] 

point #933/1084:   0%|          | 1/177350 [00:23<1156:37:36, 23.48s/it]

info retrieval:   0%|          | 1/177350 [00:23<1156:43:54, 23.48s/it] 

point #934/1084:   0%|          | 1/177350 [00:23<1156:54:13, 23.48s/it]

info retrieval:   0%|          | 1/177350 [00:23<1157:00:07, 23.49s/it] 

point #935/1084:   0%|          | 1/177350 [00:23<1157:05:52, 23.49s/it]

info retrieval:   0%|          | 1/177

point #980/1084:   0%|          | 1/177350 [00:23<1169:48:34, 23.75s/it]

info retrieval:   0%|          | 1/177350 [00:23<1169:53:25, 23.75s/it] 

point #981/1084:   0%|          | 1/177350 [00:23<1170:00:14, 23.75s/it]

info retrieval:   0%|          | 1/177350 [00:23<1170:06:31, 23.75s/it] 

point #982/1084:   0%|          | 1/177350 [00:23<1170:13:41, 23.75s/it]

info retrieval:   0%|          | 1/177350 [00:23<1170:19:17, 23.76s/it] 

point #983/1084:   0%|          | 1/177350 [00:23<1170:26:53, 23.76s/it]

info retrieval:   0%|          | 1/177350 [00:23<1170:32:09, 23.76s/it] 

point #984/1084:   0%|          | 1/177350 [00:23<1170:44:31, 23.76s/it]

info retrieval:   0%|          | 1/177350 [00:23<1170:49:47, 23.77s/it] 

point #985/1084:   0%|          | 1/177350 [00:23<1170:57:31, 23.77s/it]

info retrieval:   0%|          | 1/177350 [00:23<1171:04:02, 23.77s/it] 

point #986/1084:   0%|          | 1/177350 [00:23<1171:10:54, 23.77s/it]

info retrieval:   0%|          | 1/177

info retrieval:   0%|          | 1/177350 [00:24<1185:05:58, 24.06s/it]  

point #1031/1084:   0%|          | 1/177350 [00:24<1185:15:02, 24.06s/it]

info retrieval:   0%|          | 1/177350 [00:24<1185:20:28, 24.06s/it]  

point #1032/1084:   0%|          | 1/177350 [00:24<1185:24:48, 24.06s/it]

info retrieval:   0%|          | 1/177350 [00:24<1185:28:42, 24.06s/it]  

point #1033/1084:   0%|          | 1/177350 [00:24<1185:32:30, 24.07s/it]

info retrieval:   0%|          | 1/177350 [00:24<1185:39:43, 24.07s/it]  

point #1034/1084:   0%|          | 1/177350 [00:24<1185:47:00, 24.07s/it]

info retrieval:   0%|          | 1/177350 [00:24<1185:54:19, 24.07s/it]  

point #1035/1084:   0%|          | 1/177350 [00:24<1186:03:20, 24.08s/it]

info retrieval:   0%|          | 1/177350 [00:24<1186:09:17, 24.08s/it]  

point #1036/1084:   0%|          | 1/177350 [00:24<1186:17:59, 24.08s/it]

info retrieval:   0%|          | 1/177350 [00:24<1186:46:18, 24.09s/it]  

point #1037/1084:   0%|  

point #1081/1084:   0%|          | 1/177350 [00:24<1198:19:36, 24.32s/it]

info retrieval:   0%|          | 1/177350 [00:24<1198:33:25, 24.33s/it]  

point #1082/1084:   0%|          | 1/177350 [00:24<1198:40:53, 24.33s/it]

info retrieval:   0%|          | 1/177350 [00:24<1198:46:37, 24.33s/it]  

point #1083/1084:   0%|          | 1/177350 [00:24<1198:52:28, 24.34s/it]

info retrieval:   0%|          | 1/177350 [00:24<1198:57:46, 24.34s/it]  

point #1084/1084:   0%|          | 1/177350 [00:24<1199:02:22, 24.34s/it]

info retrieval:   0%|          | 1/177350 [00:24<1199:15:39, 24.34s/it]  

  0%|          | 1/177350 [00:25<1236:30:35, 25.10s/it]                

#### Possible modifications:
- Add more layers to the adjustment function
- Use days instead of weeks for odeint